<a href="https://colab.research.google.com/github/alejandra2826/Alejandra-Trujillo/blob/main/PRACTICA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**PRACTICA 3**

**Nombre:** Alejandra Elizabeth Trujillo Navarro
**e-mail:** alejandra.trujillo2826@alumnos.udg.mx

#**MODULES**


In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

import seaborn as sns

#**CLASSES**

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#**FUNCTIONS**

In [3]:
######################################################################
# Brownian Motion Trajectory
######################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [4]:
# Correlated Random Walk
def crw(n_steps=1000, speed=5, s_pos=[0,0], CRW_exponent = 0.5):

  CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(speed,0)

  for i in range(n_steps-1):
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': CRW_df.x_pos[i]+velocity.x, 'y_pos': CRW_df.y_pos[i]+velocity.y}])
    CRW_df = pd.concat ([CRW_df,temp_df], ignore_index=True)
  return CRW_df


In [6]:
###############################################################################################
# Levy walk
# This function generates a levy Walk
###############################################################################################
def levy_flight(n_steps=1000, init_speed = 5, s_pos=[0,0], CRW_exponent = 0.5, alpha = 1.5, beta = 0, loc=3.0):
  #Init DFs
  Levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  Levy_df = pd.concat([Levy_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(init_speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    #step size
    step_size = levy_stable.rvs(alpha=alpha, beta=beta, loc=loc)

    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': Levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos': Levy_df.y_pos[i]+(velocity.y*step_size)}])
    Levy_df = pd.concat ([Levy_df,temp_df], ignore_index=True)
  return Levy_df

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [8]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('/content/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('/content/brownian_6.csv')

In [15]:
# Seed the random number generator
rng = np.random.default_rng(42)

# Determine the number of paths and points per path
points = 1000
paths = 50

# Create the initial set of random normal draws
mu, sigma = 0.0, 1.0
Z = rng.normal(mu, sigma, (paths, points))

# Define the time step size and t-axis
interval = [0.0, 1.0]
dt = (interval[1] - interval[0]) / (points - 1)
t_axis = np.linspace(interval[0], interval[1], points)

# Use Equation brownian motion paths
W = np.zeros((paths, points))
for idx in range(points - 1):
    real_idx = idx + 1
    W[:, real_idx] = W[:, real_idx - 1] + np.sqrt(dt) * Z[:, idx]

# Create a non-zero mean and non-unit standard deviation
mu_c, sigma_c = 5.0, 2.0

# Obtain the set of final path values
temp_df = pd.DataFrame({'final_values': W[:, -1]})


# Output the mean and stdev of these final values
print(temp_df.mean(), temp_df.std())



final_values   -0.011699
dtype: float64 final_values    1.251382
dtype: float64


In [28]:
def brownian_motion(T=1, N=100, mu=0.1, sigma=0.2, S0=100):
    """
    Genera una simulación de Brownian Motion.

    Parámetros:
    - T: tiempo total de la simulación.
    - N: número de pasos.
    - mu: rendimiento promedio.
    - sigma: volatilidad.
    - S0: valor inicial.

    Devuelve un DataFrame de Pandas con los pasos de tiempo y los valores correspondientes.
    """
    dt = T / N  # Tamaño del paso de tiempo
    t = np.linspace(0, T, N+1)  # Vector de tiempo
    W = np.random.standard_normal(size=N+1)  # Proceso de Wiener
    W = np.cumsum(W) * np.sqrt(dt)  # Movimiento Browniano
    X = (mu - 0.5 * sigma**2) * t + sigma * W  # Proceso de difusión

    # Agregar el valor inicial S0 y combinar tiempo y valores en un DataFrame
    df = pd.DataFrame({'Time': t, 'Value': S0 * np.exp(X)})

    return df

# Ejemplo de uso
bm_simulation = brownian_motion(T=1, N=100, mu=0.1, sigma=0.2, S0=100)
print(bm_simulation)

     Time       Value
0    0.00   96.673304
1    0.01   95.777889
2    0.02  100.007774
3    0.03  101.109132
4    0.04  100.099636
..    ...         ...
96   0.96   99.724941
97   0.97  104.078862
98   0.98  102.600029
99   0.99  104.182017
100  1.00  104.704501

[101 rows x 2 columns]


In [30]:
# help(bm_2d)

BM_otro = bm_simulation

BM_otro.head()

# BM_otro.info()

,Time,Value
0,0.00,96.673304
1,0.01,95.777889
2,0.02,100.007774
3,0.03,101.109132
4,0.04,100.099636


In [31]:
# Get Path length calling the function
# BM con vel 3
BM_2d_df_3 = bm_simulation

# BM con vel 6
BM_2d_df_6 = bm_simulation

CRW_2d_df_9 = pd.read_csv('/content/brownian_6.csv')

In [32]:
# Plotting
# Init figure
fig_path_length = go.Figure()


# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(BM_2d_df_3))+1,
    y = BM_2d_df_3,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(BM_2d_df_6))+1,
    y = BM_2d_df_6,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(CRW_2d_df_9))+1,
    y = CRW_2d_df_9,
    name = 'path_length_CRW_6',
    showlegend = True
))
fig_path_length.show()